In [5]:
import os
import sys
import time
import requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()  # ISO 국가 코드 

BASE_URL = 'http://flupy.org/data/flags' # 국기 웹사이트
DEST_DIR = 'downloads/'                  # 저장할 디렉토리

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)

def get_flag(cc):  # url을 만들고 이진 시퀀스 반환
    cc = cc.lower()
    url = f'{BASE_URL}/{cc}/{cc}.gif'
    resp = requests.get(url)
    return resp.content

def show(text):
    print(text, end=' ', flush=True)
    
def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
    return len(cc_list)

def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == "__main__":
    main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloaded in 19.53s


### 17.1.2 concurrent.futures 로 내려받기

In [7]:
from concurrent import futures

MAX_WORKERS = 10

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    # 객체 생성. executor.shutdown(wait=True) 메서드를 호출. 
    with futures.ThreadPoolExecutor(workers) as executor:        
        res = executor.map(download_one, sorted(cc_list))
        
    return len(list(res))  # 가져온 결과의 수 반환

if __name__ == "__main__":
    main(download_many)

IN FR BD DE BR ID CN ET CD IR EG JP MX RU TR US NG PK PH VN 
20 flags downloaded in 2.70s


In [14]:
from concurrent import futures

MAX_WORKERS = 10

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    # 객체 생성. executor.shutdown(wait=True) 메서드를 호출. 
    with futures.ProcessPoolExecutor() as executor:        
        res = executor.map(download_one, sorted(cc_list))
        
    return len(list(res))  # 가져온 결과의 수 반환

if __name__ == "__main__":
    main(download_many)

Process SpawnProcess-20:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'download_one' on <module '__main__' (built-in)>
Process SpawnProcess-21:
Traceback (most recent call last):
  File "/Library

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [9]:
def download_many(cc_list):
    cc_list = cc_list[:5]
    
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list): # 국가 코드를 알파벳순으로 한다. 
            # 콜러블이 실행되도록 스케줄링하고 이 작업을 나타내는 Future 객체를 반환한다. 
            future = executor.submit(download_one, cc) 
            to_do.append(future) # as_completed()로 가져오도록 Future 객체를 모두 저장
            print(f'Scheduled for {cc}:{future}')
            
        results = []
        
        for future in futures.as_completed(to_do): # Future가 완료될 때 해당 Future 객체를 생성
            res = future.result()
            print(f'{futures} result: {res}')
            results.append(res)
            
    return len(results)

if __name__ == "__main__":
    main(download_many)

Scheduled for BR:<Future at 0x1207ebb50 state=running>
Scheduled for CN:<Future at 0x121427790 state=running>
Scheduled for ID:<Future at 0x1207bf3d0 state=running>
Scheduled for IN:<Future at 0x120809c70 state=pending>
Scheduled for US:<Future at 0x120763160 state=pending>
CN <module 'concurrent.futures' from '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/__init__.py'> result: CN
ID <module 'concurrent.futures' from '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/__init__.py'> result: ID
US <module 'concurrent.futures' from '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/__init__.py'> result: US
BR <module 'concurrent.futures' from '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/__init__.py'> result: 

## 17.4 Executor.map()  실험

In [15]:
from time import sleep, strftime
from concurrent import futures

def display(*args):  # 타임스탬프를 출력
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)

def loiter(n):  # 메시지 출력
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10 

def main():
    display('Script starting.')
    # 스레드 3개의 ThreadPoolExecutor 객체 생성
    executor = futures.ThreadPoolExecutor(max_workers=3)  
    results = executor.map(loiter, range(5))  # 5개의 작업 요청
    display('results:', results)  # executor.map이 반환한 값을 바로 출력.
    display('Waiting for individual results:')
    for i, result in enumerate(results):  # 다음 결과가 나올 때까지 루프 블로킹
        display(f'result {i}: {result}')

if __name__ == '__main__':
    main()

[23:08:20] Script starting.
[23:08:20] loiter(0): doing nothing for 0s...
[23:08:20] loiter(0): done.
[23:08:20] 	loiter(1): doing nothing for 1s...
[23:08:20] 		loiter(2): doing nothing for 2s...
[23:08:20] 			loiter(3): doing nothing for 3s...
[23:08:20] results: <generator object Executor.map.<locals>.result_iterator at 0x120847ba0>
[23:08:20] Waiting for individual results:
[23:08:20] result 0: 0
[23:08:21] 	loiter(1): done.
[23:08:21] 				loiter(4): doing nothing for 4s...
[23:08:21] result 1: 10
[23:08:22] 		loiter(2): done.
[23:08:22] result 2: 20
[23:08:23] 			loiter(3): done.
[23:08:23] result 3: 30
[23:08:25] 				loiter(4): done.
[23:08:25] result 4: 40
